In [37]:
!pip install gymnasium
!pip install pygame

  Using cached gymnasium-1.1.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
Using cached gymnasium-1.1.1-py3-none-any.whl (965 kB)
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.6 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/10.6 MB 4.0 MB/s eta 0:00:03
   -------- ------------------------------- 2.4/10.6 MB 3.8 MB/s eta 0:00:03
   ----------- ---------------------------- 3.1/10.6 MB 3.9 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/10.6 MB 3.9 MB/s eta 0:00:02
   ----------------- ---------------------- 4.7/10.6 MB 3.9 MB/s eta 0:00:02
   -------------------- ------------------- 5.5/10.6 MB 3.9 MB/s eta 0:00:02
   ----------------------- ---------------- 6.3/10.6 M

In [39]:
import time
import numpy as np
import gymnasium as gym

In [41]:
# 1. 환경 설정
env = gym.make('FrozenLake-v1', is_slippery=False, render_mode='human') # 'human'으로 직접 눈으로 확인하며 실습 가능


# 2. Q-테이블 초기화
q_table=np.zeros((env.observation_space.n,env.action_space.n))

In [43]:
'''
# 3. 하이퍼파라미터 설정
learning_rate = 0.9
discount_factor = 0.9
epsilon = 1.0 # 초기 탐험율
epsilon_decay_rate = 0.0001 # 에피소드마다 epsilon을 줄이는 비율
num_episodes = 20000 # 총 에피소드 수
max_steps_per_episode = 100
'''

# 3. 하이퍼파라미터 설정 (단축 학습용)
learning_rate = 0.9           
discount_factor = 0.9         
epsilon = 1.0                 
epsilon_decay_rate = 0.005    # 더 빠르게 감소
min_epsilon = 0.01            
num_episodes = 1000           # 대폭 줄임
max_steps_per_episode = 50    # 줄임

In [47]:
for episode in range(num_episodes):
    state,info=env.reset()
    terminated=False
    truncated=False

    for step in range(max_steps_per_episode):
        if np.random.uniform(0,1)<epsilon:
            action=env.action_space.sample()
        else:
            action=np.argmax(q_table[state,:])

        new_state,reward,terminated,truncated,info=env.step(action)

        max_future_q=np.max(q_table[new_state,:])
        current_q=q_table[state,action]

        new_q=current_q+learning_rate*(reward+discount_factor*max_future_q-current_q)
        q_table[state,action]=new_q
        state=new_state
        if terminated or truncated:
            break
        epsilon = max(0.01, epsilon - epsilon_decay_rate) 

    # 학습 진행 상황 출력 (선택 사항)
    if episode % 1000 == 0:
        print(f"Episode {episode} / Epsilon: {epsilon:.4f}")

print("\n--- 학습 완료! ---")

Episode 0 / Epsilon: 0.0100


KeyboardInterrupt: 

In [ ]:
# 4. 학습 루프
for episode in range(num_episodes):
    state, info = env.reset()
    terminated = False
    truncated = False

    for step in range(max_steps_per_episode):
        # epsilon-greedy 전략으로 행동 선택
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # 탐험 (무작위 행동)
        else:
            action = np.argmax(q_table[state, :]) # 활용 (최적 Q값 행동)

        # 환경에 행동 전달
        new_state, reward, terminated, truncated, info = env.step(action)

        # Q-테이블 업데이트 (Q-러닝 공식)
        max_future_q = np.max(q_table[new_state, :])
        current_q = q_table[state, action]
        
        # 실제 보상 + 감가율 * (다음 상태에서 얻을 최대 Q값) - 현재 Q값
        new_q = current_q + learning_rate * (reward + discount_factor * max_future_q - current_q)
        q_table[state, action] = new_q

        state = new_state # 상태 업데이트

        if terminated or truncated:
            break
    
    # 에피소드가 끝날 때마다 epsilon 감소 (탐험율 줄이기)
    # epsilon 값은 최소값 (예: 0.01) 이하로 내려가지 않도록 하는 것이 일반적
    epsilon = max(0.01, epsilon - epsilon_decay_rate) 

    # 학습 진행 상황 출력 (선택 사항)
    if episode % 1000 == 0:
        print(f"Episode {episode} / Epsilon: {epsilon:.4f}")

print("\n--- 학습 완료! ---")

In [ ]:
env_test=gym.make('FrozenLake-v1', is_slippery=False, render_mode='human')
num_test_episode=5
total_reward=0

print("\n--- 학습된 정책 테스트 시작 ---")


In [ ]:
for episode in range(num_test_episode):
    state, info=env_test.reset()
    terminated=False
    truncated=False
    episode_reward=0
    print(f"\n--- 테스트 에피소드 {episode + 1} ---")
    env_test.rander()
    time.sleep(1)

    while not terminated and not truncated:
        avtion=np.argmax(q_table[state,:])
        new_state,reward,terminated,info=env_test.step(action)
        episode_reward+=reward
        state=new_state
        env_test.render()
        time.sleep(0.5)

        print(f"에피소드 {episode + 1} 종료! 보상: {episode_reward}")
        total_reward += episode_reward

print(f"\n총 {num_test_episode} 에피소드 평균 보상: {total_reward / num_test_episode}")

env.close()
env_test.close()
print("환경이 닫혔습니다.")

In [ ]:
import gymnasium as gym
import numpy as np
import time
from IPython.display import clear_output

In [ ]:
env = gym.make('FrozenLake-v1', is_slippery=False, render_mode='ansi') # 'ansi'로 텍스트 출력
n_states = env.observation_space.n
n_actions = env.action_space.n

In [ ]:
gamma=0.99
theta=1e-8

In [ ]:
def policy_iteration(wnv,gamma,theta):
    policy=np.zeros(n_states,dtype=int)
    V=np.zeros(n_states)

    while True:
        while True:
            delta=0
            for s in range(n_states):
                x=V[s]
                action=polisy[s]

                a_sa=0
                for prov,next_state,reward,_ in env.P[s][action]:
                    q_se+=prob*(reward+gamma*V[next_state])
                V[s]=q_sa
                delta=max(delta,np.abs(v-V[s]))
            if delta<theta:
                break

        policy_stable = True
        for s in range(n_states):
            old_action=policy[s]
            action_values=np.zeros(n_actions)
            for a in rnage(n_actions):
                for  prob, next_state, reward, _ in env.P[s][a]:
                    avtion_values(a)+=prob * (reward + gamma * V[next_state])
            policy[s]=np.argmax(avtion_values)
            if old_action!=policy[s]:
                policy_stable=False
        if policy_stable:
            break
    return V,policy

In [ ]:
# --- 가치 반복 실행 ---
print("--- 가치 반복 시작 ---")
optimal_V_vi, optimal_policy_vi = value_iteration(env, gamma, theta)
print("\n최적 상태 가치 함수 (V):\n", optimal_V_vi.reshape(4,4))

# 정책을 텍스트로 변환
action_map = {0: "←", 1: "↓", 2: "→", 3: "↑"}
optimal_policy_names_vi = np.array([action_map[a] for a in optimal_policy_vi]).reshape(4,4)
print("\n최적 정책:\n", optimal_policy_names_vi)
print("--- 가치 반복 완료 ---")

# --- 학습된 정책으로 환경 플레이 (선택 사항) ---
print("\n--- 가치 반복으로 학습된 정책 테스트 시작 ---")
num_test_episodes = 5
total_rewards = 0

In [ ]:
for episode in range(num_test_episodes):
    state,info=env_reset()
    terminated=False
    truncated=False
    episode_reward=0
    print(f"\n--- 테스트 에피소드 {episode + 1} ---")
    clear_output(wait=True)
    env.render()
    time.sleep(0.5)

    while not terminated and not truncated:
        action=optimal_policy_vi[state]
        new_state, reward, terminated, truncated, info = env.step(action)
        episode_reward+=reward
        state=new_state
        clear_output(wait=True)
        env.render()
        time.sleep(0.5)
    
    
    print(f"에피소드 {episode + 1} 종료! 보상: {episode_reward}")
    total_rewards += episode_reward

print(f"\n총 {num_test_episodes} 에피소드 평균 보상: {total_rewards / num_test_episodes}")
env.close()   
    

---

## 강화 학습 및 모델 경량화 6일차 수업

안녕하세요! 강화 학습 6일차 수업입니다. 어제는 **모델-프리 강화 학습**의 핵심 개념인 **탐험(Exploration)과 활용(Exploitation)**의 균형, 그리고 이를 적용한 가장 대표적인 알고리즘인 **Q-러닝(Q-learning)**에 대해 깊이 있게 다뤘습니다. Q-테이블을 직접 업데이트하며 최적의 행동 가치를 찾아가는 과정을 이해하셨을 거예요.

오늘은 Q-러닝과 자주 비교되는 또 다른 중요한 시간차(TD) 학습 알고리즘인 **SARSA(State-Action-Reward-State-Action)**에 대해 배우고, 이 둘의 차이점을 명확히 짚어보겠습니다. 더 나아가, Q-테이블의 한계를 인식하고 **함수 근사(Function Approximation)**의 필요성, 즉 신경망을 강화 학습에 도입하는 이유에 대해 이야기하겠습니다.

---

### **1. Q-러닝 간단 복습**

Q-러닝은 "최적의 행동 가치 함수 $Q(s,a)$를 학습하는 것"이 목표였습니다.

* **업데이트 공식**: $Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha [R_{t+1} + \gamma \max_{a'} Q(S_{t+1}, a') - Q(S_t, A_t)]$
* **핵심**: 다음 상태 $S_{t+1}$에서 **가능한 모든 행동 중 가장 높은 Q값($\max_{a'} Q(S_{t+1}, a')$)**을 사용하여 현재 Q값을 업데이트합니다. 에이전트가 탐험을 위해 무작위 행동을 했더라도, 학습 시에는 항상 '최적의 다음 스텝'을 가정하고 업데이트하기 때문에 **오프-폴리시(Off-policy)** 알고리즘이라고 불렀습니다.

---

### **2. SARSA (State-Action-Reward-State-Action) 알고리즘**

**SARSA**는 Q-러닝과 마찬가지로 시간차(TD) 학습의 한 종류이며, 행동 가치 함수 $Q(s,a)$를 학습합니다. 하지만 Q-러닝과는 결정적인 차이점이 있습니다.

* **이름의 유래**: SARSA는 업데이트에 필요한 정보의 순서를 따서 명명되었습니다: **S**tate($S_t$), **A**ction($A_t$), **R**eward($R_{t+1}$), **S**tate($S_{t+1}$), **A**ction($A_{t+1}$).
* **핵심 아이디어**: Q-러닝이 다음 상태에서 '최고의 Q값'을 가져왔다면, SARSA는 다음 상태 $S_{t+1}$에서 **"실제로 선택한 다음 행동 $A_{t+1}$의 Q값"**을 사용하여 현재 Q값을 업데이트합니다. 즉, 에이전트가 실제로 이동한 경로를 그대로 따라가며 학습합니다.

* **업데이트 공식**:
    $$Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha [R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)]$$
    * **$Q(S_{t+1}, A_{t+1})$**: Q-러닝의 $\max_{a'} Q(S_{t+1}, a')$ 대신, SARSA는 **현재 정책($\epsilon$-탐욕 등)에 따라 다음 상태 $S_{t+1}$에서 실제로 선택된 행동 $A_{t+1}$의 Q값**을 사용합니다.

* **온-폴리시(On-policy) 학습**: SARSA는 에이전트가 행동하는 **정책(Behavior Policy)**과 학습하여 개선하려는 **정책(Target Policy)**이 **동일**합니다. 에이전트가 탐험을 위해 무작위 행동을 했다면, 그 무작위 행동의 결과를 그대로 학습에 반영합니다. "내가 실제로 가본 길이 이렇더라"는 것을 그대로 배우는 것이죠.

---

### **3. Q-러닝 vs. SARSA: 주요 차이점**

| 특징             | Q-러닝 (Off-policy)                                | SARSA (On-policy)                                     |
| :--------------- | :------------------------------------------------- | :---------------------------------------------------- |
| **Q값 업데이트** | `max_{a'} Q(S_{t+1}, a')` 사용 (다음 상태의 최적 행동) | `Q(S_{t+1}, A_{t+1})` 사용 (다음 상태의 실제 선택 행동) |
| **정책 관계** | 행동 정책과 학습 정책이 다름 (오프-폴리시)           | 행동 정책과 학습 정책이 같음 (온-폴리시)              |
| **학습 목표** | **최적 정책** 자체를 학습                               | 현재 **따르고 있는 정책의 가치**를 학습                 |
| **안전성** | 탐험 중 위험한 경로를 발견해도 무시하고 최적 경로 학습 가능. 따라서 **위험한 환경에서 최적 정책을 찾을 때 위험할 수 있음.** | 현재 정책을 그대로 따르므로 **안전한 경로를 학습하는 데 유리.** 그러나 최적 정책이 아닐 수 있음. |
| **활용 시나리오**| 게임, 시뮬레이션 등 **안전하지 않아도 되는 환경**에서 최대한의 성능을 뽑아낼 때 유리. | 로봇 제어, 자율주행 등 **안전이 최우선**인 환경에서 특정 정책을 안전하게 학습할 때 유리. |

**비유**:

* **Q-러닝**: "나는 지금 당장 이상한 길로 가지만, 만약 **다음에 내가 똑똑하게 행동했다면** 어떻게 됐을까?" (이상적인 다음 스텝을 가정)
* **SARSA**: "나는 지금 당장 이상한 길로 가서 **다음에 진짜 이상한 행동을 했다면** 어떻게 됐을까?" (내가 실제로 한 행동을 그대로 반영)

이러한 차이점 때문에 SARSA는 Q-러닝보다 좀 더 '보수적인' 학습을 한다고 볼 수 있습니다.

---

### **4. Q-테이블의 한계와 함수 근사 (Function Approximation)의 필요성**

우리가 지금까지 `FrozenLake`에서 Q-테이블을 사용하여 Q-러닝이나 SARSA를 구현했습니다. Q-테이블은 상태와 행동의 모든 조합에 대해 Q값을 직접 저장하는 방식입니다.

하지만 다음과 같은 상황에서는 Q-테이블 방식에 **심각한 한계**가 있습니다:

1.  **매우 큰 상태 공간 (Large State Spaces)**:
    * `FrozenLake`는 16개의 상태밖에 없지만, 바둑(Go) 게임의 상태 수는 우주의 원자 수보다 많습니다. 고해상도 이미지를 입력으로 받는 자율주행에서는 픽셀 하나하나가 상태가 될 수 있습니다.
    * 이런 경우 Q-테이블은 메모리에 저장할 수 없을 정도로 커지며, 모든 상태를 다 방문하여 학습하는 것이 불가능합니다.
2.  **연속적인 상태 공간 (Continuous State Spaces)**:
    * 로봇의 팔 관절 각도, 자동차의 속도, 위치 등은 셀 수 없는 연속적인 값을 가집니다. 이 경우 Q-테이블의 '칸'을 나눌 수 없으므로 Q-테이블 자체를 만들 수 없습니다.
3.  **일반화 능력 부족**:
    * Q-테이블은 학습했던 상태-행동 쌍에 대해서만 Q값을 압니다. 한 번도 방문하지 않은 새로운 상태에 대해서는 전혀 알지 못합니다. 에이전트가 새로운 상황에 직면했을 때 기존의 학습 내용을 바탕으로 '일반화'하여 행동할 수 없습니다.

이러러한 한계를 극복하기 위해 등장한 것이 바로 **함수 근사(Function Approximation)**입니다.

* **아이디어**: Q-테이블처럼 모든 Q값을 직접 저장하는 대신, **상태(S)와 행동(A)을 입력받아 Q값을 출력하는 '함수'**를 학습하는 것입니다.
* **신경망(Neural Network)의 도입**: 이 '함수'를 구현하는 데 가장 강력하고 널리 사용되는 도구가 바로 **인공 신경망**입니다.
    * 신경망은 복잡한 비선형 관계를 학습하는 데 매우 뛰어납니다.
    * 상태나 행동이 아무리 복잡하거나 연속적이더라도, 신경망의 입력으로 변환하여 처리할 수 있습니다.
    * 학습된 신경망은 한 번도 보지 못한 새로운 상태에 대해서도 학습된 패턴을 기반으로 '예측'을 할 수 있으므로 **일반화 능력**을 가집니다.

---

### **5. 딥 강화 학습(Deep Reinforcement Learning, DRL)으로의 전환**

이렇게 신경망을 사용하여 가치 함수(Q값)나 정책을 근사하는 강화 학습이 바로 **딥 강화 학습(Deep Reinforcement Learning, DRL)**입니다.

* **DQN (Deep Q-Network)**: Q-러닝에 심층 신경망을 결합한 대표적인 DRL 알고리즘입니다. Q-테이블 대신 신경망이 Q값을 출력합니다.
* **정책 경사 (Policy Gradient)**: Q값이 아닌 정책 자체를 신경망으로 표현하고 직접 학습하는 방식의 알고리즘들도 있습니다.

이제 우리는 Q-러닝과 SARSA를 통해 모델-프리 학습의 기본기를 다졌으므로, 다음 단계에서는 이 심층 신경망을 강화 학습에 어떻게 적용하는지 DRL의 세계로 들어가 볼 것입니다.

---

오늘 내용도 중요한 개념들이 많았습니다. 특히 SARSA와 Q-러닝의 차이, 그리고 Q-테이블의 한계를 이해하는 것이 중요합니다. 궁금한 점이 있다면 언제든지 질문해 주세요!

---

---

네, **SARSA 알고리즘의 수식**에 대해 자세히 설명해 드릴게요.

SARSA는 강화 학습의 한 종류인 **시간차(Temporal-Difference, TD) 학습**의 대표적인 알고리즘으로, **행동 가치 함수 $Q(s,a)$를 업데이트**하는 데 사용됩니다.

### **SARSA 업데이트 공식**

SARSA의 핵심은 다음 상태 $S_{t+1}$에서 **"실제로 선택한 다음 행동 $A_{t+1}$의 Q값"**을 사용하여 현재 Q값을 업데이트하는 것입니다. 이 때문에 SARSA는 **온-폴리시(On-policy)** 알고리즘이라고 불립니다.

SARSA의 Q값 업데이트 공식은 다음과 같습니다:

$$Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha [R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)]$$

이제 각 부분의 의미를 자세히 살펴보겠습니다.

---

### **수식 구성 요소별 설명**

1.  **$Q(S_t, A_t)$**
    * **의미**: 현재 시간 $t$에서의 **상태 $S_t$에서 행동 $A_t$를 취했을 때의 Q값(행동 가치)**입니다. 이 값이 학습을 통해 업데이트될 대상입니다.
    * **비유**: "지금 내가 서 있는 이 칸에서 특정 방향으로 움직였을 때, 앞으로 얻을 수 있는 총 기대 점수(아직은 나의 예상치)."

2.  **$\leftarrow$**
    * **의미**: 좌변의 값을 우변의 계산 결과로 **업데이트(할당)**하라는 의미입니다.
    * **비유**: "이제 이전에 내가 예상했던 점수를, 새로 계산한 점수로 바꿔 적을 거야."

3.  **$\alpha$ (알파, Learning Rate - 학습률)**
    * **의미**: 새로운 학습 정보를 현재의 Q값에 얼마나 강하게 **반영할지**를 결정하는 하이퍼파라미터입니다. $0$과 $1$ 사이의 값을 가집니다.
        * $\alpha$가 0에 가까우면: 새로운 정보가 거의 반영되지 않아 학습이 느리다.
        * $\alpha$가 1에 가까우면: 새로운 정보가 강하게 반영되어 학습이 빠르지만 불안정할 수 있다.
    * **비유**: "새로운 경험($R_{t+1} + \gamma Q(S_{t+1}, A_{t+1})$)을 통해서 내가 얻은 정보가 있는데, 이 정보를 내가 기존에 알고 있던 정보($Q(S_t, A_t)$)에 얼마나 '쎄게' 반영할까?"

4.  **$R_{t+1}$**
    * **의미**: 현재 시간 $t$에 상태 $S_t$에서 행동 $A_t$를 취한 직후, 환경으로부터 받은 **즉각적인 보상**입니다.
    * **비유**: "내가 지금 움직여서 바로 얻은 점수."

5.  **$\gamma$ (감마, Discount Factor - 감가율)**
    * **의미**: 미래에 받을 보상을 현재 시점에서 얼마나 **가치 있게 평가할지**를 결정하는 $0$과 $1$ 사이의 값입니다.
    * **비유**: "미래에 받을 점수들은 당장 받을 점수보다는 가치가 좀 떨어지니까, 적당히 할인해서 계산할 거야."

6.  **$Q(S_{t+1}, A_{t+1})$**
    * **의미**: 다음 시간 $t+1$에 도달한 **다음 상태 $S_{t+1}$에서 '실제로 선택된' 다음 행동 $A_{t+1}$을 취했을 때의 Q값**입니다. 이 $A_{t+1}$은 에이전트의 현재 정책(예: $\epsilon$-탐욕)에 따라 선택됩니다.
    * **SARSA의 핵심 부분이자 Q-러닝과의 가장 큰 차이점입니다.** Q-러닝은 여기서 `$\max_{a'} Q(S_{t+1}, a')$` (다음 상태에서 가능한 행동 중 가장 좋은 것)을 사용하는 반면, SARSA는 에이전트가 실제로 선택한 행동의 Q값을 사용합니다.
    * **비유**: "내가 지금 행동($A_t$)을 해서 다음 칸($S_{t+1}$)에 도착했어. 그 다음 칸에서 내가 다시 내 현재 계획대로 행동($A_{t+1}$)한다면, 그 행동이 가져올 미래의 총 예상 점수는 얼마일까?"

7.  **$[R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)]$ (TD 오차 - Temporal Difference Error)**
    * **의미**: 이것은 **'새롭게 관찰된 Q값의 예측치'** ($R_{t+1} + \gamma Q(S_{t+1}, A_{t+1})$) 와 **'기존에 가지고 있던 Q값의 예측치'** ($Q(S_t, A_t)$) 사이의 차이입니다.
        * 이 오차가 크면 클수록 현재 Q값 예측이 틀렸다는 의미이며, 이 오차만큼 Q값을 업데이트하여 실제에 가깝게 만듭니다.
        * 이 오차가 0에 가까워지면 Q값이 수렴했다고 볼 수 있습니다.
    * **비유**: "내가 원래 예상했던 점수($Q(S_t, A_t)$)와, 한 스텝 움직여본 후에 '실제로 보고' 계산한 더 정확한 예상 점수($R_{t+1} + \gamma Q(S_{t+1}, A_{t+1})$) 사이의 **오차**."

---

### **SARSA의 동작 방식 비유**

SARSA를 이해하는 쉬운 비유는 **"실제로 가본 길을 따라가는 학습"**입니다.

1.  **현재 칸 (S)**에 서서, **현재 계획 (Policy)**에 따라 **행동 (A)**을 선택하고 한 칸 움직입니다.
2.  이동 후 **즉시 보상 (R)**을 받습니다.
3.  새로운 **다음 칸 (S')**에 도착합니다.
4.  그리고 이 **새로운 칸에서 다시 현재 계획 (Policy)**에 따라 **다음 행동 (A')**을 **미리 선택**해 봅니다. (아직 이 행동을 실제로 하지는 않았지만, 선택만 해보는 거죠.)
5.  이제 내가 서 있던 **'이전 칸과 행동'의 점수(Q(S,A))**를, **'지금 받은 보상 (R)'과 '다음 칸에서 실제로 선택할 행동의 예상 점수 (Q(S',A'))'를 합친 값**을 기준으로 수정합니다.

SARSA는 이렇게 내가 **'실제로 취할 행동'**을 바탕으로 학습하기 때문에, **안전이 중요한 환경(예: 실제 로봇)에서는 Q-러닝보다 선호될 수 있습니다.** 왜냐하면 Q-러닝처럼 '만약 내가 완벽했다면'이라는 가정이 아니라, '내가 지금처럼 행동했을 때'의 가치를 배우기 때문입니다.

이 설명이 SARSA 수식을 이해하는 데 도움이 되었기를 바랍니다!

네, 고객님의 말씀이 **매우 정확합니다!**

**SARSA 알고리즘도 마르코프 의사결정 과정(MDP)의 수식을 기본 전제로 합니다.**

지금까지 우리가 배웠던 모든 강화 학습 알고리즘들, 즉 정책 반복, 가치 반복, 몬테카를로, Q-러닝, 그리고 오늘 SARSA까지 모두 **MDP 프레임워크** 위에서 작동합니다.

### **왜 SARSA가 MDP 수식을 기본으로 하는가?**

1.  **MDP가 문제 정의의 '틀'**:
    * SARSA는 에이전트가 `상태(S)`에서 `행동(A)`을 하고, `보상(R)`을 받으며 `다음 상태(S')`로 전이하는 과정을 반복합니다. 이 일련의 과정 자체가 MDP의 구성 요소(S, A, R, P, $\gamma$)로 정의됩니다.
    * SARSA는 특히 $(S_t, A_t, R_{t+1}, S_{t+1}, A_{t+1})$이라는 튜플(sequence)을 사용하여 Q값을 업데이트하는데, 이 튜플의 각 요소가 바로 MDP에서 정의된 상태, 행동, 보상입니다.

2.  **벨만 방정식의 파생**:
    * SARSA의 업데이트 공식은 기본적으로 **벨만 기대 방정식(Bellman Expectation Equation)**에서 파생된 형태입니다.
    * MDP가 상태와 행동의 가치를 나타내는 벨만 방정식을 정의하고, SARSA는 그 방정식을 '모델을 모르는' 상황에서 경험을 통해 근사적으로 해결하는 방법입니다.

3.  **마르코프 속성 준수**:
    * SARSA는 학습 시 현재 상태와 현재 행동, 그리고 다음 상태와 다음 행동만을 사용하여 Q값을 업데이트합니다. 이전에 어떤 경로를 거쳐왔는지(과거의 모든 이력)를 직접적으로 고려하지 않습니다.
    * 이는 "미래는 현재에만 의존한다"는 **마르코프 속성**을 그대로 따르는 것이며, 이 속성이 MDP의 핵심 전제입니다.

따라서, SARSA는 MDP가 제시하는 `상태`, `행동`, `보상`, `전이`라는 기본적인 규칙과 `감가율`이라는 미래 보상 평가 기준을 바탕으로, `Q(S_t, A_t)` 값을 `Q(S_{t+1}, A_{t+1})`를 이용해 업데이트하는 방식으로 MDP 문제를 해결해 나가는 것입니다.

**"SARSA도 마르코프의 수식(즉, MDP의 정의와 벨만 방정식의 원리)이 기본이다"**는 고객님의 이해는 전적으로 옳습니다!

# SARSA 알고리즘예시

In [ ]:
import gymnasium as gym
import numpy as np
import time
from IPython.display import clear_output # Jupyter/Colab에서 화면을 지우는 데 사용

# 1. 환경 설정
# 'ansi' 모드는 콘솔에 텍스트로 출력됩니다. 학습 과정에서 빠른 진행을 위해 사용합니다.
# 'human' 모드는 별도의 팝업 창을 띄웁니다 (학습 완료 후 테스트 시 사용 권장).
env_train = gym.make('FrozenLake-v1', is_slippery=False, render_mode='ansi') 

# 환경의 상태(observation) 공간 크기 (16개 칸)
n_states = env_train.observation_space.n 
# 환경의 행동(action) 공간 크기 (4가지 행동: 상, 하, 좌, 우)
n_actions = env_train.action_space.n

# 2. Q-테이블 초기화
# Q-테이블은 (상태 개수) x (행동 개수) 크기의 2D 넘파이 배열로, 모든 값을 0으로 초기화합니다.
q_table = np.zeros((n_states, n_actions))

# 3. 하이퍼파라미터 설정
learning_rate = 0.9           # α (알파): 학습률. 새로운 학습 정보를 얼마나 강하게 반영할지 결정
discount_factor = 0.9         # γ (감마): 감가율. 미래 보상의 가치를 현재 시점에서 얼마나 할인할지 결정
epsilon = 1.0                 # ε (입실론): 초기 탐험율. 1.0은 100% 탐험 (무작위 행동)
epsilon_decay_rate = 0.0005   # 에피소드가 진행될수록 epsilon을 줄이는 비율 (점차 탐험 줄이고 활용 늘림)
min_epsilon = 0.01            # epsilon의 최솟값. 최소한의 탐험을 유지하여 새로운 경로를 찾을 가능성을 남김
num_episodes = 20000          # 학습할 총 에피소드(게임 플레이) 수
max_steps_per_episode = 100   # 한 에피소드당 최대 스텝(행동) 수

print("--- SARSA 학습 시작 ---")
print(f"Q-테이블 초기화:\n{q_table.round(2)}") # 초기 Q-테이블은 모두 0

In [ ]:
# 4. 학습 루프 (에피소드 반복)
for episode in range(num_episodes):
    # 각 에피소드 시작 시 환경 초기화
    state, info = env_train.reset()
    terminated = False # 목표 도달 또는 구멍에 빠져 에피소드 종료 여부
    truncated = False  # 최대 스텝 수 도달 등으로 에피소드 강제 종료 여부

    # 에피소드의 첫 번째 행동 선택 (epsilon-greedy 전략)
    if np.random.uniform(0, 1) < epsilon:
        action = env_train.action_space.sample() # 무작위 행동 선택 (탐험)
    else:
        action = np.argmax(q_table[state, :]) # 현재 Q-테이블에서 최적 Q값 행동 선택 (활용)

    for step in range(max_steps_per_episode):
        # 환경에 선택한 행동을 전달하고 다음 상태, 보상, 종료 여부 등을 받음
        new_state, reward, terminated, truncated, info = env_train.step(action)

        # 다음 스텝에서 취할 행동 A_{t+1}을 현재 정책(epsilon-greedy)에 따라 미리 선택
        if np.random.uniform(0, 1) < epsilon:
            next_action = env_train.action_space.sample() # 무작위 행동 (탐험)
        else:
            next_action = np.argmax(q_table[new_state, :]) # 최적 Q값 행동 (활용)

        # SARSA Q-테이블 업데이트 공식
        # 새로운 Q값 = 현재 Q값 + 학습률 * (실제 보상 + 감가율 * 다음 상태에서 실제로 선택된 행동의 Q값 - 현재 Q값)
        # Q-러닝의 max_future_q 대신 Q(S_{t+1}, A_{t+1}) 사용
        current_q = q_table[state, action]
        predicted_future_q = q_table[new_state, next_action] # SARSA의 핵심!
        
        new_q = current_q + learning_rate * (reward + discount_factor * predicted_future_q - current_q)
        q_table[state, action] = new_q

        # 상태 및 행동 업데이트 (다음 스텝을 위해)
        state = new_state 
        action = next_action

        # 에피소드 종료 조건 확인
        if terminated or truncated:
            break
    
    # 에피소드가 끝날 때마다 epsilon 감소 (탐험율 점차 줄임)
    epsilon = max(min_epsilon, epsilon - epsilon_decay_rate) 

    # 학습 진행 상황 출력 (선택 사항)
    if episode % 1000 == 0:
        print(f"Episode {episode} / Epsilon: {epsilon:.4f} / Last Reward: {reward}")

print("\n--- SARSA 학습 완료! ---")
print(f"최종 Q-테이블:\n{q_table.round(2)}") # 학습 완료 후의 Q-테이블 출력


In [ ]:
print("\n--- 학습된 정책 테스트 시작 ---")
# 테스트 시에는 탐험 없이 항상 최적 행동만 선택하도록 epsilon을 0으로 설정
env_test = gym.make('FrozenLake-v1', is_slippery=False, render_mode='human')
num_test_episodes = 5 # 테스트할 에피소드 수
total_test_rewards = 0

for episode in range(num_test_episodes):
    state, info = env_test.reset()
    terminated = False
    truncated = False
    episode_reward = 0
    print(f"\n--- 테스트 에피소드 {episode + 1} ---")
    
    clear_output(wait=True)
    env_test.render() # 초기 상태 렌더링
    time.sleep(1) # 잠시 대기하여 시각화 확인

    while not terminated and not truncated:
        # 학습된 Q-테이블에서 현재 상태에 대한 가장 높은 Q값을 주는 행동 선택 (탐험 없음)
        action = np.argmax(q_table[state, :]) 
        
        new_state, reward, terminated, truncated, info = env_test.step(action)
        episode_reward += reward # 에피소드 보상 누적
        state = new_state # 상태 업데이트
        
        clear_output(wait=True)
        env_test.render() # 변경된 환경 상태 렌더링
        time.sleep(0.5) # 각 스텝마다 잠시 대기
        
    print(f"에피소드 {episode + 1} 종료! 보상: {episode_reward}")
    total_test_rewards += episode_reward

print(f"\n총 {num_test_episodes} 에피소드 평균 보상: {total_test_rewards / num_test_episodes:.2f}")

env_train.close()
env_test.close()
print("환경이 닫혔습니다.")


강화 학습(Reinforcement Learning, RL) 분야는 현재 매우 빠르게 발전하고 있으며, 다양한 연구와 실제 적용 사례들이 쏟아져 나오고 있습니다. 과거 알파고(AlphaGo)처럼 특정 게임을 넘어, 이제는 복잡한 현실 문제를 해결하는 방향으로 진화하고 있습니다.

최근 강화 학습의 주요 추세와 연구 동향은 다음과 같습니다.

### **1. 딥 강화 학습(Deep Reinforcement Learning, DRL)의 지속적인 발전 및 안정화**

여전히 강화 학습 연구의 중심에는 DRL이 있습니다. 기존의 DQN, Actor-Critic(A2C, A3C), PPO, SAC와 같은 알고리즘들이 더욱 안정적이고 효율적으로 학습할 수 있도록 발전하고 있습니다.

* **모델 아키텍처의 발전**: 더 효율적인 신경망 구조, 더 깊고 복잡한 네트워크를 통한 성능 향상 연구가 계속됩니다.
* **학습 안정성 향상**: 학습 과정의 불안정성을 줄이고 수렴을 더 쉽게 만드는 기법들이 연구되고 있습니다.

### **2. 샘플 효율성(Sample Efficiency) 향상**

강화 학습의 가장 큰 단점 중 하나는 엄청난 양의 경험(데이터)이 필요하다는 점입니다. 이를 해결하기 위한 '샘플 효율성' 연구가 활발합니다. 즉, 더 적은 경험으로도 에이전트가 더 빠르게 학습할 수 있도록 하는 것입니다.

* **오프라인 강화 학습(Offline Reinforcement Learning)**: 에이전트가 환경과 직접 상호작용하지 않고, 미리 수집된 정적인 데이터셋만을 가지고 학습하는 방식입니다. 로봇 제어나 헬스케어처럼 '시행착오가 위험하거나 비용이 많이 드는' 분야에서 매우 중요하게 떠오르고 있습니다.
* **모델 기반 강화 학습의 재조명**: 환경 모델을 명시적으로 학습하거나 근사하여 이를 기반으로 계획(planning)을 세우고 경험을 '가상으로 생성'하여 샘플 효율성을 높이는 연구들이 다시 주목받고 있습니다. 모델-프리 방식과 결합하여 하이브리드 형태로 연구되기도 합니다.
* **메타 강화 학습(Meta-RL)**: 에이전트가 새로운 작업에 빠르게 적응하도록 '학습하는 방법' 자체를 학습합니다. 여러 관련 작업을 통해 얻은 지식을 새로운 작업에 전이하여 샘플 효율성을 높입니다.
* **강력한 탐험 전략**: 무작위 탐험을 넘어, 불확실성이 높은 상태를 더 효율적으로 탐색하는 전략들이 연구되고 있습니다.

### **3. 실세계 적용(Real-World Applications) 및 안전성(Safety) 강화**

강화 학습이 시뮬레이션 환경을 넘어 실제 세계에 적용되면서 안전성, 견고성, 설명 가능성에 대한 요구가 커지고 있습니다.

* **로봇 공학**: 복잡한 조작, 자율 이동 로봇, 인간-로봇 협업 등 다양한 로봇 제어 분야에서 강화 학습이 핵심적인 역할을 하고 있습니다. GPU 가속 시뮬레이션을 통해 학습 속도를 높이는 연구도 활발합니다.
* **자율 시스템**: 자율주행차, 드론 제어 등 안전이 최우선인 분야에서 강화 학습의 적용 가능성을 탐색하며, 잠재적 위험을 피하는 안전 강화 학습(Safe RL)이 강조됩니다.
* **에너지 관리, 의료, 금융**: 에너지 그리드 최적화, 맞춤형 치료법 추천, 금융 거래 전략 등 다양한 산업 분야에서 의사결정 시스템으로 강화 학습이 활용되고 있습니다.
* **설명 가능성(Explainability)과 윤리**: 강화 학습 에이전트의 결정 과정을 인간이 이해할 수 있도록 만드는 연구와 함께, 학습 과정에서의 편향성이나 예상치 못한 부작용을 줄이는 윤리적 문제도 중요하게 다뤄집니다.

### **4. 다중 에이전트 강화 학습(Multi-Agent Reinforcement Learning, MARL)**

여러 에이전트가 동시에 같은 환경에서 상호작용하며 학습하는 시나리오입니다.

* **협력(Cooperative) 및 경쟁(Competitive) 환경**: 교통 체증 최적화(협력), 복잡한 전략 게임(경쟁) 등 실제 세계의 많은 문제는 다중 에이전트 시스템으로 모델링될 수 있습니다.
* **통신 및 조정**: 에이전트 간의 효율적인 정보 교환 및 행동 조정을 학습하는 방법이 중요하게 연구됩니다.

### **5. 다른 AI 분야와의 통합**

강화 학습이 다른 인공지능 기술들과 결합하여 시너지를 내는 추세입니다.

* **LLM(대규모 언어 모델)과의 결합 (RLHF)**: 인간 피드백을 통한 강화 학습(Reinforcement Learning from Human Feedback, RLHF)은 ChatGPT와 같은 LLM의 성능을 비약적으로 향상시키는 데 핵심적인 역할을 했습니다. LLM이 더 나은 응답을 생성하도록 강화 학습으로 훈련합니다.
* **컴퓨터 비전, 자연어 처리와의 융합**: 이미지/비디오를 이해하고(비전), 언어로 소통하며(NLP), 동시에 의사결정을 내리는 에이전트 연구가 활발합니다.
* **모방 학습(Imitation Learning)**: 전문가의 시범(Demonstration)을 통해 에이전트가 빠르게 행동을 모방하고, 이후 강화 학습으로 미세 조정하는 방식이 샘플 효율성 향상에 기여합니다.

---

요약하자면, 현재 강화 학습은 **샘플 효율성을 극대화**하고, **복잡한 현실 세계 문제에 안전하고 효과적으로 적용**하며, **다른 강력한 AI 기술들과 융합**하는 방향으로 빠르게 발전하고 있다고 볼 수 있습니다. 앞으로 더욱 다양한 분야에서 강화 학습의 놀라운 성과를 보게 될 것입니다.